# Indice espectrales SAVI en GEE

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee,{"color" : "00FF11"},name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Seleccion imagen Landsat RS

In [6]:
# Seleccionar una imagen Landsat8
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_003069_20190808")\
                  .select(["B2","B3","B4","B5","B6","B7"],
                          ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001)\
                  .reproject(crs = "EPSG:32719", scale = 30)\
                  .clip(zona_ee)

In [7]:
# Seleccionar una imagen Sentinel2
img_S2 = ee.Image("COPERNICUS/S2_SR/20190808T145729_20190808T145732_T19LCF")\
                  .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],
                          ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001)\
                  .clip(zona_ee)

## Consultar nombre bandas y proyección

In [8]:
# Nombre de las bandas
print("Nombre bandas L8: ",img_L8.bandNames().getInfo())
print("Nombre bandas S2: ",img_S2.bandNames().getInfo())

Nombre bandas L8:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
Nombre bandas S2:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']


In [9]:
# Consultar Proyeccion
print("Proyeccion L8: ",img_L8.projection().getInfo())
print("Proyeccion S2: ",img_S2.select(['BLUE']).projection().getInfo())

Proyeccion L8:  {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion S2:  {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [10, 0, 300000, 0, -10, 8600020]}


In [10]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [12]:
# Visualizar 
Map.addLayer(img_L8, viz_es, name = "Landsat8 RS")
Map.addLayer(img_S2, viz_es, name = "Sentinel2 RS")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Determinar SAVI

In [13]:
# Funcion para SAVI
def Calc_SAVI(image):
    savi = image.expression("float(nir - red)/(nir + red + L)*(1 + L)", {
        'red': image.select('RED'),
        'nir': image.select('NIR'),
        'L': 0.5
    })
    return savi.rename("SAVI")

In [14]:
# Calcular indices con funcion
savi_L8 = Calc_SAVI(img_L8)
savi_S2 = Calc_SAVI(img_S2)

In [15]:
# Simbologia de SAVI
viz_savi = {
    'palette': ['FFFFFF', 'CE7E45',
                'DF923D', 'F1B555',
                'FCD163', '99B718',
                '74A901', '66A000',
                '529400', '3E8601',
                '207401', '056201',
                '004C00', '023B01',
                '012E01', '011D01',
                '011301'],
    'min': -0.4,
    'max': 1,
    'bands': "SAVI"
}

In [16]:
# Visualizar 
Map.addLayer(savi_L8, viz_savi, name = "Landsat8 SAVI")
Map.addLayer(savi_S2, viz_savi, name = "Sentinel2 SAVI")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Exportar Imagen

In [17]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [18]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [19]:
# Exportar imagen directorio
geemap.ee_export_image(savi_L8, filename="L8_savi.tif", scale=30, region=geometria, file_per_band=False)
# geemap.ee_export_image(savi_S2, filename="S2_savi.tif", scale=10, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_savi.tif
Generating URL ...
An error occurred while downloading.
Total request size (98259616 bytes) must be less than or equal to 33554432 bytes.


In [20]:
# Exportar en google drive
geemap.ee_export_image_to_drive(savi_L8, description='savi_L8', folder='GEE_Python', region=geometria, scale=30)
geemap.ee_export_image_to_drive(savi_S2, description='savi_S2', folder='GEE_Python', region=geometria, scale=10)

Exporting savi_L8 ...
Exporting savi_S2 ...
